# Test basiques

In [ ]:
# nombre de transactions où le wallet reçoit un token

SELECT COUNT(db.block_timestamp) AS distinct_transaction_count
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.to_address = {address}

# nombre de transactions où le wallet émet un token

SELECT COUNT(db.block_timestamp) AS distinct_transaction_count
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.from_address = {address}


In [ ]:
#Première transaction de réception d'un token

SELECT db.block_timestamp
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.to_address = {address}
ORDER BY db.block_timestamp ASC
LIMIT 1

#Première transaction d'émission d'un token

SELECT db.block_timestamp
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.from_address = {address}
ORDER BY db.block_timestamp ASC
LIMIT 1

#Dernière transaction de réception d'un token

SELECT db.block_timestamp
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.to_address = {address}
ORDER BY db.block_timestamp DESC
LIMIT 1

#Dernière transaction d'émission d'un token

SELECT db.block_timestamp
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.from_address = {address}
ORDER BY db.block_timestamp DESC
LIMIT 1

In [ ]:
#nombre de tokens différents reçus par le wallet

SELECT COUNT(DISTINCT db.token_address) AS distinct_token_count
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.to_address = {address}

#nombre de tokens différents émis par le wallet

SELECT COUNT(DISTINCT db.token_address) AS distinct_token_count
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.from_address = {address}

In [ ]:
# nombre de partenaires de réception de tokens par le wallet

SELECT COUNT(DISTINCT db.to_address) AS distinct_token_recievers
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.from_address = {address}

# nombre de partenaires d'émission de tokens par le wallet

SELECT COUNT(DISTINCT db.from_address) AS distinct_token_senders
FROM `bigquery-public-data.crypto_ethereum.token_transfers` AS db
WHERE db.to_address = {address}

# Token analysis - 1 address

In [ ]:
# token_value info

SELECT token_address, COUNT(CAST(value AS FLOAT64)) AS total_count, SUM(CAST(value AS FLOAT64)) AS total_value
FROM (
  SELECT token_address, value
  FROM `bigquery-public-data.crypto_ethereum.token_transfers`
  WHERE from_address = {address}

  UNION ALL

  SELECT token_address, value
  FROM `bigquery-public-data.crypto_ethereum.token_transfers`
  WHERE to_address = {address}
) AS token_request_uery
GROUP BY token_address;

In [ ]:
# Time_transaction info

SELECT block_timestamp
FROM (
    SELECT block_timestamp
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    WHERE from_address = {address}
      AND token_address = {token_address}
    ORDER BY block_timestamp ASC
    LIMIT 1
) AS earliest_sent_query

UNION ALL

SELECT block_timestamp
FROM (
    SELECT block_timestamp
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    WHERE from_address = {address}
      AND token_address = {token_address}
    ORDER BY block_timestamp DESC
    LIMIT 1
) AS latest_sent_query

UNION ALL

SELECT block_timestamp
FROM (
    SELECT block_timestamp
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    WHERE to_address = {address}
      AND token_address = {token_address}
    ORDER BY block_timestamp ASC
    LIMIT 1
) AS earliest_received_query

UNION ALL 

SELECT block_timestamp
FROM (
    SELECT block_timestamp
    FROM `bigquery-public-data.crypto_ethereum.token_transfers`
    WHERE to_address = {address}
      AND token_address = {token_address}
    ORDER BY block_timestamp DESC
    LIMIT 1
) AS latest_received_query

# Requetes utiles

In [ ]:
#Activité wallet

SELECT tt.token_address,
       COUNT(CAST(tt.value AS FLOAT64)) AS total_count,
       SUM(CAST(tt.value AS FLOAT64)) AS total_value,
       COUNT(CASE WHEN tt.from_address = {address} THEN 1 END) AS sent_transaction_count,
       SUM(CASE WHEN tt.from_address = {address} THEN CAST(tt.value AS FLOAT64) ELSE 0 END) AS sent_value,
       COUNT(CASE WHEN tt.to_address = {address} THEN 1 END) AS received_transaction_count,
       SUM(CASE WHEN tt.to_address = {address} THEN CAST(tt.value AS FLOAT64) ELSE 0 END) AS received_value,
       MIN(est.block_timestamp) AS earliest_sent,
       MAX(lst.block_timestamp) AS latest_sent,
       MIN(ert.block_timestamp) AS earliest_received,
       MAX(lrt.block_timestamp) AS latest_received

FROM (
  -- Combine token transfers for from and to addresses
  SELECT token_address, value, from_address, to_address, block_timestamp
  FROM bigquery-public-data.crypto_ethereum.token_transfers tt
  WHERE tt.from_address = {address} OR tt.to_address = {address}
) AS tt
LEFT JOIN (
  -- Earliest sent transfers for each token_address
  SELECT token_address, MIN(block_timestamp) AS block_timestamp
  FROM bigquery-public-data.crypto_ethereum.token_transfers
  WHERE from_address = {address}
  GROUP BY token_address
) AS est ON tt.token_address = est.token_address

LEFT JOIN (
  -- Latest sent transfers for each token_address
  SELECT token_address, MAX(block_timestamp) AS block_timestamp
  FROM bigquery-public-data.crypto_ethereum.token_transfers
  WHERE from_address = {address}
  GROUP BY token_address
) AS lst ON tt.token_address = lst.token_address

LEFT JOIN (
  -- Earliest recieved transfers for each token_address
  SELECT token_address, MIN(block_timestamp) AS block_timestamp
  FROM bigquery-public-data.crypto_ethereum.token_transfers
  WHERE to_address = {address}
  GROUP BY token_address
) AS ert ON tt.token_address = ert.token_address

LEFT JOIN (
  -- Latest recieved transfers for each token_address
  SELECT token_address, MAX(block_timestamp) AS block_timestamp
  FROM bigquery-public-data.crypto_ethereum.token_transfers
  WHERE to_address = {address}
  GROUP BY token_address
) AS lrt ON tt.token_address = lrt.token_address

GROUP BY tt.token_address;

In [ ]:
#historique changements balance

WITH token_transfers AS (
  SELECT
    block_timestamp,
    token_address,
    from_address AS wallet_address,
    -CAST(value AS FLOAT64) AS value
  FROM
    bigquery-public-data.crypto_ethereum.token_transfers
  WHERE
    CAST(value AS FLOAT64) != 0
  UNION ALL
  SELECT
    block_timestamp,
    token_address,
    to_address AS wallet_address,
    CAST(value AS FLOAT64) AS value
  FROM
    bigquery-public-data.crypto_ethereum.token_transfers
  WHERE
    CAST(value AS FLOAT64) != 0
),
daily_balances AS (
  SELECT
    DATE(block_timestamp) AS day,
    wallet_address,
    token_address,
    SUM(value) AS daily_value
  FROM
    token_transfers
  GROUP BY
    day,
    wallet_address,
    token_address
),
filtered_balances AS (
  SELECT
    db.day AS timestamp,
    db.wallet_address,
    db.token_address,
    CAST(SUM(db.daily_value) OVER (PARTITION BY db.token_address ORDER BY db.day ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) / 1e14 AS FLOAT64) AS balance
  FROM
    daily_balances db
  LEFT JOIN
    bigquery-public-data.crypto_ethereum.contracts c
  ON
    db.wallet_address = c.address
  WHERE
    c.address IS NULL
    AND db.day >= '2024-01-01' AND db.day < '2024-04-01'
)
SELECT
  timestamp,
  wallet_address,
  token_address,
  balance
FROM
  filtered_balances
ORDER BY
  token_address, timestamp DESC;